<a href="https://colab.research.google.com/github/Deeks900/GenAIBootcamp/blob/main/2_Text_Processing_Logistic_Regression_and_Boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression and Boosting Algorithms

© Data Trainers LLC. GPL v 3.0.

**Author:** Axel Sirota


## Predicting a Single Categorical Response
---



### Installing stuff

In [2]:
!pip install --upgrade textblob spacy gensim swifter 'numpy<1.26'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 61.0 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16505 sha256=4a505203c429857a7718ac462dbd479feecaf43b190b8b4b7ae4bb8807316593
  Stored in directory: /root/.cache/pip/wheels/ef/7f/bd/9bed48f078f3ee1fa75e0b29b6e0335ce1cb03a38d3443b3a3
Succes

In [1]:
!python -m textblob.download_corpora lite
!python -m spacy download en_core_web_sm


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
Finished.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 83.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB         # Naive Bayes
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
import torch
import spacy
import gensim
import warnings
import nltk
warnings.filterwarnings('ignore')
nltk.download('punkt')
textblob_tokenizer = lambda x: TextBlob(x).words


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
%%writefile get_data.sh
if [ ! -f yelp.csv ]; then
  wget -O yelp.csv https://www.dropbox.com/scl/fi/dr6xmgw59kliq74gcd340/yelp.csv?rlkey=la6ue9a899v54f04eu92lbmlx&st=kjg7pmvm&dl=0
fi

Writing get_data.sh


In [ ]:
!bash get_data.sh

--2025-04-23 08:19:50--  https://www.dropbox.com/scl/fi/dr6xmgw59kliq74gcd340/yelp.csv?rlkey=la6ue9a899v54f04eu92lbmlx
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3f0f81978723a0cce5241d56ce.dl.dropboxusercontent.com/cd/0/inline/CoXoxyv9Za2GHt2GrGbU4PB8CZLPIsF0lO8eRl49Id4gRkYdL5xYi54fQ1umKYbxYWclnizCoex3shhtGTD5IpqukJ3BQGEcLlNWz-OY_7tk5oxY4toorGU_6WwNEsKNZIwAP4aLtqZD0qkXUg__Mhut/file# [following]
--2025-04-23 08:19:51--  https://uc3f0f81978723a0cce5241d56ce.dl.dropboxusercontent.com/cd/0/inline/CoXoxyv9Za2GHt2GrGbU4PB8CZLPIsF0lO8eRl49Id4gRkYdL5xYi54fQ1umKYbxYWclnizCoex3shhtGTD5IpqukJ3BQGEcLlNWz-OY_7tk5oxY4toorGU_6WwNEsKNZIwAP4aLtqZD0qkXUg__Mhut/file
Resolving uc3f0f81978723a0cce5241d56ce.dl.dropboxusercontent.com (uc3f0f81978723a0cce5241d56ce.dl.dropboxusercontent.com)... 162.125.2.15, 2620:100:6

In [ ]:
# Read yelp.csv into a DataFrame.
path = './yelp.csv'
yelp = pd.read_csv(path)
# Create a new DataFrame that only contains the 5-star and 1-star reviews.
yelp_best_worst = yelp[ (yelp.stars == 1) | (yelp.stars == 5) ]

# Define X and y.
X = yelp_best_worst.text
y = yelp_best_worst.stars

# Split the new DataFrame into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

<a id="using-logistic-regression-for-classification"></a>
## Using Logistic Regression for Classification
---



In [ ]:
# Fit a logistic regression model to predict stars from text

logreg = LogisticRegression()

logreg.fit(X,y)


ValueError: could not convert string to float: 'My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.\n\nDo yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I\'ve ever had.  I\'m pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.\n\nWhile EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I\'ve ever had.\n\nAnyway, I can\'t wait to go back!'

Of course this simply fails, we need to preprocess the text, convert it into a Tensor format and then and only then we can use models!

### Converting text to vectors

In [ ]:
import re
nltk.download('stopwords')
my_stopwords = nltk.corpus.stopwords.words('english')
word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'


def preprocess_text(text, should_join=True):
    text = ' '.join(word.lower() for word in textblob_tokenizer(text))
    text = re.sub(r'http\S+', '', text) # remove http links
    text = re.sub(r'bit.ly/\S+', '', text) # rempve bitly links
    text = text.strip('[link]') # remove [links]
    text = re.sub('['+my_punctuation + ']+', ' ', text) # remove punctuation
    text = re.sub('\s+', ' ', text) #remove double spacing
    text = re.sub(r"[^a-zA-Z.,&!?]+", r" ", text) # only normal characters
    # text_token_list = [word for word in text.split(' ')
    #                         if word not in my_stopwords] # remove stopwords
    # text_token_list = [word_rooter(word) if '#' not in word else word
    #                     for word in text_token_list] # apply word rooter
    # text = ' '.join(text)
    if should_join:
      return ' '.join(gensim.utils.simple_preprocess(text))
    else:
      return gensim.utils.simple_preprocess(text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
preprocess_text("This is the best day of my life. Isn't it? Я тобі сама ", should_join=False)

['this', 'is', 'the', 'best', 'day', 'of', 'my', 'life', 'is', 'it']

In [ ]:
# Apply the preprocessing to the dataset
import swifter
X_preprocessed = X.swifter.apply(preprocess_text)

Pandas Apply:   0%|          | 0/4086 [00:00<?, ?it/s]

In [ ]:
X_preprocessed[0]

'my wife took me here on my birthday for breakfast and it was excellent the weather was perfect which made sitting outside overlooking their grounds an absolute pleasure our waitress was excellent and our food arrived quickly on the semi busy saturday morning it looked like the place fills up pretty quickly so the earlier you get here the better do yourself favor and get their bloody mary it was phenomenal and simply the best ve ever had pretty sure they only use ingredients from their garden and blend them fresh when you order it it was amazing while everything on the menu looks excellent had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious it came with pieces of their griddled bread with was amazing and it absolutely made the meal complete it was the best toast ve ever had anyway ca wait to go bac'

How do we pass from text to numbers? With tokenizers.

In [ ]:
# Implement this method
def get_maximum_review_length(srs):
    maximum = 0
    for review in srs:
      if len(review.split()) > maximum:
        maximum = len(review.split())
    return maximum


maximum = get_maximum_review_length(X_preprocessed)

In [ ]:
print(f'The maximum review was {maximum} words long')

The maximum review was 879 words long


In [ ]:
!pip install pytorch-nlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 4.1 MB/s eta 0:00:00


In [ ]:
import itertools
from torchnlp.encoders import LabelEncoder

# Use the LabelEncoder to use as Tokenizer that transforms words into indices.
# Hint: You may need to create first a list of all words in your corpus

# WAY 1 : Double loop
# words = set()
# for list_of_words in X_preprocessed.apply(lambda x: x.split()):
#   for word in list_of_words:
#     words.add(word)
# ids_from_words = LabelEncoder(list(words))

ids_from_words = LabelEncoder(list(itertools.chain.from_iterable(X_preprocessed.swifter.apply(lambda x: x.split()))), reserved_labels=['[UNK]'], unknown_index=0)



Pandas Apply:   0%|          | 0/4086 [00:00<?, ?it/s]

In [ ]:
ids_from_words.vocab_size

18937

In [ ]:
ids_from_words.batch_encode(["breakfast"])

tensor([9])

In [ ]:
ids_from_words.decode(ids_from_words.encode("breakfast"))


'breakfast'

In [ ]:
def text_from_ids(ids):
  return ids_from_words.batch_decode(ids)

In [ ]:
def ids_from_text(text):
  return ids_from_words.batch_encode(text)

In [ ]:
ids = ids_from_text('Only you can prevent forest fires'.lower().split())
ids

tensor([  60,   44,  112, 6583, 6589, 8309])

In [ ]:
text_from_ids(ids)




['only', 'you', 'can', 'prevent', 'forest', 'fires']

In [ ]:
def pad_sequence_of_tokens(x, maxlen, unk_token='[UNK]'):
  if len(x)<maxlen:
    x.extend([unk_token]*(maxlen-len(x)))
  return x

In [ ]:
def get_tensor(x, maximum=maximum):
  padding = (0, maximum-ids_from_text(x).shape[-1])
  return torch.squeeze(F.pad(ids_from_text(x), padding, "constant", 0).to(torch.long))


In [ ]:
import torch.nn.functional as F
# Very useful method
def get_ids_tensor(srs):

  processed = srs.swifter.apply(lambda x: pad_sequence_of_tokens(preprocess_text(x, should_join=False), maxlen=maximum))
  result = processed.swifter.apply(get_tensor).to_list()
  return torch.stack(result)


In [ ]:
all_ids = get_ids_tensor(srs=X_preprocessed.reset_index(drop=True))
all_ids

Pandas Apply:   0%|          | 0/4086 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4086 [00:00<?, ?it/s]

tensor([[   1,    2,    3,  ...,    0,    0,    0],
        [  99,  100,  101,  ...,    0,    0,    0],
        [ 206,  207,   10,  ...,    0,    0,    0],
        ...,
        [ 965, 1636,  182,  ...,    0,    0,    0],
        [4458,  685,  169,  ...,    0,    0,    0],
        [   0,  203, 2108,  ...,    0,    0,    0]])

In [ ]:
all_ids.shape

torch.Size([4086, 879])

In [ ]:
# Split the all_ids into.a train a test sets
X_train, X_test, y_train, y_test = train_test_split(all_ids, y, random_state=1)

### Using Logistic Regression

In [ ]:

# Train a Logistic Regression on X_train and give the accuracy
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


0.764187866927593


## Using Boosting Algorithms and other things

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

clf = GradientBoostingClassifier(n_estimators=50, learning_rate=0.5)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.27      0.09      0.14       184
           5       0.83      0.94      0.88       838

    accuracy                           0.79      1022
   macro avg       0.55      0.52      0.51      1022
weighted avg       0.72      0.79      0.75      1022



In [ ]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=50, learning_rate=0.5)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       184
           5       0.82      1.00      0.90       838

    accuracy                           0.82      1022
   macro avg       0.41      0.50      0.45      1022
weighted avg       0.67      0.82      0.74      1022



In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=50)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.62      0.03      0.05       184
           5       0.82      1.00      0.90       838

    accuracy                           0.82      1022
   macro avg       0.72      0.51      0.48      1022
weighted avg       0.79      0.82      0.75      1022



## Multiclass Classification

Just check in the estimators, most support multiclass classification.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
X, y = load_iris(return_X_y=True)
clf = LogisticRegression(random_state=0, multi_class='multinomial').fit(X, y)
clf.predict(X[:2, :])
clf.predict_proba(X[:2, :])
clf.score(X, y)

### **Homework**: Try to perform the stars classification with Logistic Regression but without filtering only for 5 and 1 stars.